<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/nl_whisper.(new1)ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-gi99k0kb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-gi99k0kb
  Resolved https://github.com/huggingface/transformers to commit 5b7a225f2502840f4066b1b39ef53411ea4d6e90
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9050429 sha256=648c135d0c1765f30592a116c07903a397213b53c2dabb76eedb4b17b7f35bbc
  Stored in directory: /tmp/pip-ephem-wheel-cache-y_epctzc/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:


In [6]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [7]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [8]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "nl", split="test")
#column_name = "audio, normalized_text"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for facebook/voxpopuli contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/voxpopuli
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  wa

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1137
})

In [10]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using c

In [12]:
model = whisper.load_model("large")

In [13]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [14]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [15]:
text

,0
0,en daar die eigen middelen het ook mogelijk ma...
1,je hebt een zeer goede en sterke europese gedr...
2,tegen die achtergrond vindt het voornemen dat ...
3,er is wel een duidelijke europese uitdaging al...
4,nog dramatischer is de situatie in libanon. da...
...,...
1132,de lijst van recente misdaden tegen christenen...
1133,ik wil graag dat de boeren op hun eigen benen ...
1134,extra regels zullen het geschonden consumenten...
1135,ik vind ook dat de wetenschap zoveel mogelijk ...


In [16]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [17]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,en daar die eigen middelen het ook mogelijk ma...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00738525390625, 0.00439453125, 0.0005798339...",16000
1,female,je hebt een zeer goede en sterke europese gedr...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000213623046875, 0.000152587890625, -0.0001...",16000
2,male,tegen die achtergrond vindt het voornemen dat ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.02880859375, -0.034271240234375, -0.033355...",16000
3,male,er is wel een duidelijke europese uitdaging al...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, 0.000152587890625, -6.103515...",16000
4,male,nog dramatischer is de situatie in libanon. da...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.0001220703125, -0.00018...",16000
...,...,...,...,...,...
1132,male,de lijst van recente misdaden tegen christenen...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00103759765625, 0.00103759765625, 0.0012207...",16000
1133,male,ik wil graag dat de boeren op hun eigen benen ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.193695068359375, -0.7314453125, -0.4067687...",16000
1134,male,extra regels zullen het geschonden consumenten...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0008544921875, -0.00079345703125, -0.00012...",16000
1135,male,ik vind ook dat de wetenschap zoveel mogelijk ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001861572265625, 0.00115966796875, 0.00265...",16000


In [18]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,en daar die eigen middelen het ook mogelijk ma...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00738525390625, 0.00439453125, 0.0005798339...",16000
2,male,tegen die achtergrond vindt het voornemen dat ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.02880859375, -0.034271240234375, -0.033355...",16000
3,male,er is wel een duidelijke europese uitdaging al...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, 0.000152587890625, -6.103515...",16000
4,male,nog dramatischer is de situatie in libanon. da...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.0001220703125, -0.00018...",16000
5,male,wij als parlement moeten ook niet doen alsof w...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.010894775390625, 0.0115966796875, 0.0105895...",16000
...,...,...,...,...,...
1132,male,de lijst van recente misdaden tegen christenen...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00103759765625, 0.00103759765625, 0.0012207...",16000
1133,male,ik wil graag dat de boeren op hun eigen benen ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.193695068359375, -0.7314453125, -0.4067687...",16000
1134,male,extra regels zullen het geschonden consumenten...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0008544921875, -0.00079345703125, -0.00012...",16000
1135,male,ik vind ook dat de wetenschap zoveel mogelijk ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001861572265625, 0.00115966796875, 0.00265...",16000


In [19]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
1,female,je hebt een zeer goede en sterke europese gedr...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000213623046875, 0.000152587890625, -0.0001...",16000
9,female,ik denk dat het een oogopener moet zijn of ik ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.076171875, -0.060577392578125, -0.03576660...",16000
10,female,deze soort kan tot dertig jaar oud worden.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00701904296875, 0.002227783203125, -0.00909...",16000
13,female,voorzitter of je nu voor kernernergie bent of ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.010009765625, 0.00469970703125, 0.005584716...",16000
14,female,willen we dat mensen tweederangsburgers zijn v...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0162353515625, 0.031524658203125, 0.0357360...",16000
...,...,...,...,...,...
1125,female,dat kan natuurlijk.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000244140625, -0.0001220703125, -6.1035156...",16000
1126,female,laten we elkaar vooral aan deze gemaakte afspr...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0003662109375, 0.000457763671875, 0.000701...",16000
1128,female,je ziet in veel landen dat er een quotasysteem...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.039276123046875, -0.052520751953125, -0.07...",16000
1129,female,ja dit parlement debatteert met emotie en nee ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.01861572265625, -0.033599853515625, -0.048...",16000


In [20]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="nl",fp16=False)
    return result["text"]

In [21]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [22]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.2
Processed 2 files, WER: 0.13636363636363635
Processed 3 files, WER: 0.2
Processed 4 files, WER: 0.3684210526315789
Processed 5 files, WER: 0.21428571428571427
Processed 6 files, WER: 0.18421052631578946
Processed 7 files, WER: 0.1111111111111111
Processed 8 files, WER: 0.35
Processed 9 files, WER: 1.0
Processed 10 files, WER: 0.2
Processed 11 files, WER: 0.26666666666666666
Processed 12 files, WER: 0.3333333333333333
Processed 13 files, WER: 0.1702127659574468
Processed 14 files, WER: 0.13333333333333333
Processed 15 files, WER: 1.0
Processed 16 files, WER: 0.3333333333333333
Processed 17 files, WER: 0.2222222222222222
Processed 18 files, WER: 0.6666666666666666
Processed 19 files, WER: 0.1875
Processed 20 files, WER: 0.35714285714285715
Processed 21 files, WER: 0.2916666666666667
Processed 22 files, WER: 0.16666666666666666
Processed 23 files, WER: 0.4
Processed 24 files, WER: 0.9
Processed 25 files, WER: 0.08333333333333333
Processed 26 files, WER: 0.33333

In [23]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.3076923076923077
Processed 2 files, WER: 1.0
Processed 3 files, WER: 0.375
Processed 4 files, WER: 0.36
Processed 5 files, WER: 0.25
Processed 6 files, WER: 0.35294117647058826
Processed 7 files, WER: 0.25
Processed 8 files, WER: 0.2222222222222222
Processed 9 files, WER: 0.5
Processed 10 files, WER: 0.3
Processed 11 files, WER: 0.42857142857142855
Processed 12 files, WER: 0.16666666666666666
Processed 13 files, WER: 0.25
Processed 14 files, WER: 0.2
Processed 15 files, WER: 0.24242424242424243
Processed 16 files, WER: 0.21739130434782608
Processed 17 files, WER: 0.2727272727272727
Processed 18 files, WER: 0.36363636363636365
Processed 19 files, WER: 0.1875
Processed 20 files, WER: 0.3058823529411765
Processed 21 files, WER: 1.0
Processed 22 files, WER: 1.0
Processed 23 files, WER: 0.2564102564102564
Processed 24 files, WER: 0.22580645161290322
Processed 25 files, WER: 0.7777777777777778
Processed 26 files, WER: 0.3
Processed 27 files, WER: 0.23076923076923078

In [24]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [25]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)

    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1

    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")
    # Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.2, Precision: 0.8333333333333334, Recall: 0.8, F1: 0.816326530612245
Processed 2 files, WER: 0.3076923076923077, Precision: 0.75, Recall: 0.84, F1: 0.7924528301886793
Processed 3 files, WER: 0.13636363636363635, Precision: 0.8529411764705882, Recall: 0.8529411764705882, F1: 0.8529411764705882
Processed 4 files, WER: 0.2, Precision: 0.8, Recall: 0.8, F1: 0.8000000000000002
Processed 5 files, WER: 0.3684210526315789, Precision: 0.625, Recall: 0.5882352941176471, F1: 0.6060606060606061
Processed 6 files, WER: 0.21428571428571427, Precision: 0.8571428571428571, Recall: 0.9230769230769231, F1: 0.888888888888889
Processed 7 files, WER: 0.18421052631578946, Precision: 0.7941176470588235, Recall: 0.7941176470588235, F1: 0.7941176470588235
Processed 8 files, WER: 0.1111111111111111, Precision: 0.9375, Recall: 0.8823529411764706, F1: 0.9090909090909091
Processed 9 files, WER: 0.35, Precision: 0.631578947368421, Recall: 0.6666666666666666, F1: 0.6486486486486486
Processe